# Imports

In [1]:
from pathlib import Path
from numpy.random import seed
from tensorflow.random import set_seed
from time import strftime

import os
import numpy as np
import tensorflow.compat.v1 as tf

In [2]:
# Set the seeds
seed(888)
set_seed(404)

from PIL import Image

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [3]:
MNIST_PATH = Path('MNIST')
MNIST_PATH.mkdir(parents=True, exist_ok=True)
LOGGING_PATH = Path('tensorboard_mnist_digit_logs')
TRAIN_PATH = Path('train')
VALIDATION_PATH = Path('validation')

X_TRAIN_PATH = MNIST_PATH / "digit_xtrain.csv"
X_TEST_PATH = MNIST_PATH / "digit_xtest.csv"
Y_TRAIN_PATH = MNIST_PATH / "digit_ytrain.csv"
Y_TEST_PATH = MNIST_PATH / "digit_ytest.csv"

MNIST_IMAGE = MNIST_PATH / "test_img.png"

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * CHANNELS

# Get the Data

In [4]:
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [7]:
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

In [8]:
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

In [9]:
x_train_all.shape

(60000, 784)

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [11]:
y_train_all.shape

(60000,)

In [12]:
x_test.shape

(10000, 784)

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Processing

In [14]:
# Re-scale
x_train_all, x_test = x_train_all /255.0, x_test / 255.0

### Convert target values to one-hot encoding

In [15]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [17]:
y_train_all.shape

(60000, 10)

In [18]:
y_test = np.eye(NR_CLASSES)[y_test]

In [19]:
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [20]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [21]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [22]:
x_train.shape

(50000, 784)

In [23]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

In [24]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

## Neural Network Architecture

### Hyperparameters

In [25]:
nr_epochs = 5
learning_rate = 1e-4

n_hidden1 = 512
n_hidden2 = 64

In [26]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [27]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [28]:
folder_name = f'model1_{strftime("%H%M")}'
directory = Path.joinpath(LOGGING_PATH, folder_name)
directory.mkdir(parents=True, exist_ok=True)

# Loss, Optimisation & Metrics

#### Defining Loss Function

In [29]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [30]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [31]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [32]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [33]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [34]:
sess = tf.Session()

#### Setup Filewriter and Merge Summaries


In [35]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(Path.joinpath(directory, TRAIN_PATH).mkdir(parents=True, exist_ok=True))
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(Path.joinpath(directory, VALIDATION_PATH).mkdir(parents=True, exist_ok=True))

#### Initialize all the variables

In [36]:
init = tf.global_variables_initializer()
sess.run(init)

### Batching the Data

In [37]:
size_of_batch = 1000

In [38]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [39]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training Loop

In [40]:
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ======================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.37700000405311584
Epoch 1 	| Training Accuracy = 0.7070000171661377
Epoch 2 	| Training Accuracy = 0.7910000085830688
Epoch 3 	| Training Accuracy = 0.8320000171661377
Epoch 4 	| Training Accuracy = 0.8379999995231628
Done training!


# Make a Prediction

In [41]:
img = Image.open(MNIST_IMAGE)
img

In [42]:
bw = img.convert('L')

In [43]:
img_array = np.invert(bw)

In [44]:
img_array.shape

(28, 28)

In [45]:
test_img = img_array.ravel()

In [46]:
test_img.shape

(784,)

In [47]:
# Tensorflow session
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img]})

In [48]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [6]


# Testing and Evaluation

In [49]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})

In [50]:
print(f'Accuracy for test image is {test_accuracy:0.2%}')

Accuracy for test image is 80.46%


## Reset for the Next Run

In [51]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()